In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import logging
import time
import random
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict

# Configure logging
logging.basicConfig(level=logging.INFO)

class ValorantStatsScraper:
    def __init__(self):
        self.base_url = "https://www.vlr.gg"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

    def fetch_player_profile_url(self, player_name: str) -> str:
        """Fetch the profile URL of a player based on their name."""
        search_url = f"{self.base_url}/search/?q={player_name.replace(' ', '%20')}&type=players"
        response = requests.get(search_url, headers=self.headers)
        
        logging.info(f"Searching for player: {player_name} at {response.url}")
        if response.status_code != 200:
            logging.error(f"Error fetching search results for {player_name}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        player_link = soup.find('a', class_='search-item')

        if player_link:
            profile_url = self.base_url + player_link['href']
            logging.info(f"Profile URL found: {profile_url}")
            return profile_url
        else:
            logging.warning(f"No profile found for {player_name}.")
            return None

    def extract_player_stats(self, profile_url: str) -> Dict:
        """Extract statistics for a player from their profile."""
        full_url = f"{profile_url}?timespan=all"
        response = requests.get(full_url, headers=self.headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        player_name = soup.find('h1', class_='wf-title').text.strip() if soup.find('h1', class_='wf-title') else 'Unknown'
        player_team = soup.find('div', class_='wf-title-med').text.strip() if soup.find('div', class_='wf-title-med') else 'No Team'

        stats_table = soup.find('table', class_='wf-table')
        player_stats = []

        if stats_table:
            for row in stats_table.find('tbody').find_all('tr'):
                cells = row.find_all('td')
                if len(cells) >= 2:
                    agent_name = cells[0].find('img')['alt'] if cells[0].find('img') else 'Unknown'
                    stats = [cell.text.strip() for cell in cells[1:]]
                    player_stats.append({
                        'Agent': agent_name,
                        'Usage': stats[0],  # Usage percentage
                        'ACS': stats[3],    # Average Combat Score
                        'K/D Ratio': stats[1],  # Kill/Death Ratio
                        'Win Rate': stats[2]  # Win Rate
                    })

            return {
                'Name': player_name,
                'Team': player_team,
                'Stats': player_stats
            }
        else:
            logging.warning(f"Statistics table not found for player: {player_name}")
            return None

def save_stats_to_csv(player_data, filename='sentinels.csv'):
    """Append player statistics to a CSV file."""
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write headers if the file is empty
        if file.tell() == 0:
            writer.writerow([
                'Name', 'Agent', 'pick_percent', 'avg_score', 'kd_ratio', 'win_rate'  # Updated headers
            ])

        for stat in player_data['Stats']:
            writer.writerow([
                player_data['Name'],
                stat['Agent'],
                stat['Usage'],  # Usage is now renamed to pick_percent
                stat['ACS'],    # ACS is now renamed to avg_score
                stat['K/D Ratio'],  # Added K/D Ratio
                stat['Win Rate']     # Added Win Rate
            ])

def gather_player_data(player_name, scraper):
    """Gather and save data for a specific player."""
    profile_url = scraper.fetch_player_profile_url(player_name)
    if profile_url:
        player_stats = scraper.extract_player_stats(profile_url)
        if player_stats:
            save_stats_to_csv(player_stats)

def collect_and_store_player_data(player_names: List[str], concurrent=False):
    """Collect and store data for a list of players."""
    scraper = ValorantStatsScraper()
    
    logging.info(f"Starting data collection for {len(player_names)} players...")
    
    if concurrent:
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(gather_player_data, name, scraper) for name in player_names]
            for future in futures:
                future.result()  # Wait for all threads to finish
    else:
        for name in player_names:
            logging.info(f"Processing player: {name}")
            gather_player_data(name, scraper)
            time.sleep(random.uniform(1, 2))  # Respectful scraping

    logging.info("Data collection completed.")

if __name__ == "__main__":
    try:
        names_input = input("Enter player names (comma-separated): ")
        player_names = [name.strip() for name in names_input.split(',')]

        concurrent_mode = input("Would you like to scrape concurrently? (y/n): ").lower() == 'y'

        collect_and_store_player_data(player_names, concurrent=concurrent_mode)

    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")

In [ ]:
import requests
from lxml import html
import pandas as pd  # Import pandas for CSV functionality

def fetch_agent_stats(rank):
    # Construct the URL for the specified rank
    url = f"https://blitz.gg/valorant/stats/agents?type=general&mode=competitive&rank={rank}&map=all"
    
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the response content using lxml
        tree = html.fromstring(response.content)
        
        # Extract agent rows (each agent is in a <tr> tag)
        agent_rows = tree.xpath('//tr[@data-has-link="true"]')
        
        # Initialize a list to store agent stats
        agent_stats = []
        
        # Iterate over each agent row
        for row in agent_rows:
            # Extract agent name
            agent_name = row.xpath('.//td[2]//span/text()')[0] if row.xpath('.//td[2]//span/text()') else 'N/A'
        
            # Extract K/D Ratio
            kd_ratio = row.xpath('.//td[3]//div[@class="cell type-body2-form--bold"]/text()')[0] if row.xpath('.//td[3]//div[@class="cell type-body2-form--bold"]/text()') else 'N/A'
        
            # Extract Win Rate
            win_rate = row.xpath('.//td[5]//div[@class="cell type-body2-form--bold"]/text()')[0] if row.xpath('.//td[5]//div[@class="cell type-body2-form--bold"]/text()') else 'N/A'
        
            # Extract Pick %
            pick_percent = row.xpath('.//td[6]//div[@class="cell type-body2-form--bold"]/text()')[0] if row.xpath('.//td[6]//div[@class="cell type-body2-form--bold"]/text()') else 'N/A'
        
            # Extract Average Score
            avg_score = row.xpath('.//td[7]//div[@class="cell type-body2-form--bold"]/text()')[0] if row.xpath('.//td[7]//div[@class="cell type-body2-form--bold"]/text()') else 'N/A'
        
            # Extract Matches
            matches = row.xpath('.//td[8]//div[@class="cell type-body2-form--bold"]/text()')[0] if row.xpath('.//td[8]//div[@class="cell type-body2-form--bold"]/text()') else 'N/A'
        
            # Collect the agent data into a dictionary
            agent_stats.append({
                "name": agent_name,
                "kd_ratio": kd_ratio,
                "win_rate": win_rate,
                "pick_percent": pick_percent,
                "avg_score": avg_score,
                "matches": matches
            })
        
        # Return the collected stats for each agent
        return agent_stats
    else:
        print(f"Failed to retrieve data for rank {rank}. Status code: {response.status_code}")
        return None

def save_to_csv(agent_stats, filename):
    # Create a DataFrame from the agent stats
    df = pd.DataFrame(agent_stats)
    
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Fetch and save Radiant agent stats
radiant_rank = 27  
radiant_agent_stats = fetch_agent_stats(radiant_rank)

print("Radiant Stats:")
if radiant_agent_stats:
    for stats in radiant_agent_stats:
        print(stats)
    save_to_csv(radiant_agent_stats, 'radiant.csv')  # Save to radiant.csv

# Fetch and save Gold agent stats
print("Gold Stats:")
gold_rank = 12
gold_agent_stats = fetch_agent_stats(gold_rank)
if gold_agent_stats:
    for stats in gold_agent_stats:
        print(stats)
    save_to_csv(gold_agent_stats, 'gold.csv')  # Save to gold.csv

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import logging
import time
import random
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict

# Configure logging
logging.basicConfig(level=logging.INFO)

class ValorantStatsScraper:
    def __init__(self):
        self.base_url = "https://www.vlr.gg"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

    def fetch_player_profile_url(self, player_name: str) -> str:
        """Fetch the profile URL of a player based on their name."""
        search_url = f"{self.base_url}/search/?q={player_name.replace(' ', '%20')}&type=players"
        response = requests.get(search_url, headers=self.headers)
        
        logging.info(f"Searching for player: {player_name} at {response.url}")
        if response.status_code != 200:
            logging.error(f"Error fetching search results for {player_name}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        player_link = soup.find('a', class_='search-item')

        if player_link:
            profile_url = self.base_url + player_link['href']
            logging.info(f"Profile URL found: {profile_url}")
            return profile_url
        else:
            logging.warning(f"No profile found for {player_name}.")
            return None

    def extract_player_stats(self, profile_url: str) -> Dict:
        """Extract statistics for a player from their profile."""
        full_url = f"{profile_url}?timespan=all"
        response = requests.get(full_url, headers=self.headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        player_name = soup.find('h1', class_='wf-title').text.strip() if soup.find('h1', class_='wf-title') else 'Unknown'
        player_team = soup.find('div', class_='wf-title-med').text.strip() if soup.find('div', class_='wf-title-med') else 'No Team'

        stats_table = soup.find('table', class_='wf-table')
        player_stats = []

        if stats_table:
            for row in stats_table.find('tbody').find_all('tr'):
                cells = row.find_all('td')
                if len(cells) >= 2:
                    agent_name = cells[0].find('img')['alt'] if cells[0].find('img') else 'Unknown'
                    stats = [cell.text.strip() for cell in cells[1:]]
                    player_stats.append({
                        'Agent': agent_name,
                        'Usage': stats[0],
                        'ACS': stats[3],  # Only keep ACS
                    })

            return {
                'Name': player_name,
                'Team': player_team,
                'Stats': player_stats
            }
        else:
            logging.warning(f"Statistics table not found for player: {player_name}")
            return None

def save_stats_to_csv(player_data, filename='player_stats.csv'):
    """Append player statistics to a CSV file."""
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write headers if the file is empty
        if file.tell() == 0:
            writer.writerow([
                'Name', 'Agent', 'pick_percent', 'avg_score'  # Updated headers
            ])

        for stat in player_data['Stats']:
            writer.writerow([
                player_data['Name'],
                stat['Agent'],
                stat['Usage'],  # Usage is now renamed to pick_percent
                stat['ACS']     # ACS is now renamed to avg_score
            ])

def gather_player_data(player_name, scraper):
    """Gather and save data for a specific player."""
    profile_url = scraper.fetch_player_profile_url(player_name)
    if profile_url:
        player_stats = scraper.extract_player_stats(profile_url)
        if player_stats:
            save_stats_to_csv(player_stats)

def collect_and_store_player_data(player_names: List[str], concurrent=False):
    """Collect and store data for a list of players."""
    scraper = ValorantStatsScraper()
    
    logging.info(f"Starting data collection for {len(player_names)} players...")
    
    if concurrent:
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(gather_player_data, name, scraper) for name in player_names]
            for future in futures:
                future.result()  # Wait for all threads to finish
    else:
        for name in player_names:
            logging.info(f"Processing player: {name}")
            gather_player_data(name, scraper)
            time.sleep(random.uniform(1, 2))  # Respectful scraping

    logging.info("Data collection completed.")

if __name__ == "__main__":
    try:
        names_input = input("Enter player names (comma-separated): ")
        player_names = [name.strip() for name in names_input.split(',')]

        concurrent_mode = input("Would you like to scrape concurrently? (y/n): ").lower() == 'y'

        collect_and_store_player_data(player_names, concurrent=concurrent_mode)

    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_win_rate_comparison(gold_file='gold.csv', radiant_file='radiant.csv'):
    # Read the gold and radiant CSV files
    gold_data = pd.read_csv(gold_file)
    radiant_data = pd.read_csv(radiant_file)

    # Check the data type of 'win_rate' and convert to string if necessary
    gold_data['win_rate'] = gold_data['win_rate'].astype(str)
    radiant_data['win_rate'] = radiant_data['win_rate'].astype(str)

    # Convert 'win_rate' to numeric, handling any errors
    gold_data['win_rate'] = pd.to_numeric(gold_data['win_rate'].str.replace('%', ''), errors='coerce') / 100
    radiant_data['win_rate'] = pd.to_numeric(radiant_data['win_rate'].str.replace('%', ''), errors='coerce') / 100

    # Group by agent name and calculate the mean win rate
    gold_win_rates = gold_data.groupby('name')['win_rate'].mean().reset_index()
    radiant_win_rates = radiant_data.groupby('name')['win_rate'].mean().reset_index()

    # Merge the two DataFrames on agent name
    combined_win_rates = pd.merge(gold_win_rates, radiant_win_rates, on='name', suffixes=('_gold', '_radiant'))

    # Prepare data for plotting
    agents = combined_win_rates['name']
    gold_rates = combined_win_rates['win_rate_gold']
    radiant_rates = combined_win_rates['win_rate_radiant']

    # Set the bar width
    bar_width = 0.35
    x = range(len(agents))

    # Plotting the win rates
    plt.figure(figsize=(12, 6))
    bars1 = plt.bar(x, gold_rates, width=bar_width, label='Gold Win Rate', color='gold')
    bars2 = plt.bar([p + bar_width for p in x], radiant_rates, width=bar_width, label='Radiant Win Rate', color='blue')

    plt.xlabel('Agent')
    plt.ylabel('Win Rate')
    plt.title('Win Rate Comparison of Agents Between Gold and Radiant Players')
    plt.xticks([p + bar_width / 2 for p in x], agents, rotation=45, ha='right')  # Center the x-ticks

    # Adjust the legend position
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Place legend outside the plot area

    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.grid(axis='y')  # Add grid lines for better readability

    # Show the plot
    plt.show()

if __name__ == "__main__":
    plot_win_rate_comparison()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re  # Import regular expressions module

def plot_average_sentinel_pick_percentage(input_filename='sentinels.csv'):
    # Load the sentinels CSV file
    sentinel_df = pd.read_csv(input_filename)

    # Print the columns to check for the correct column names
    print("Sentinel Data Columns:", sentinel_df.columns)

    # Clean the 'pick_percent' column to extract only the percentage
    sentinel_df['pick_percent'] = sentinel_df['pick_percent'].astype(str).str.extract(r'(\d+\.?\d*)%')[0]  # Extract percentage
    sentinel_df['pick_percent'] = pd.to_numeric(sentinel_df['pick_percent'], errors='coerce')  # Convert to float

    # Group by agent name and calculate the mean pick percentage
    average_pick_percentage = sentinel_df.groupby('Agent')['pick_percent'].mean().reset_index()

    # Sort by average pick percentage to get the top agents
    top_agents = average_pick_percentage.nlargest(20, 'pick_percent')

    # Sort the top agents alphabetically
    top_agents = top_agents.sort_values(by='Agent')

    # Prepare data for plotting
    agents = top_agents['Agent']
    pick_frequencies = top_agents['pick_percent']

    # Plotting the Average Pick Percentages for Top Agents in Sentinels
    plt.figure(figsize=(12, 6))
    bars = plt.bar(agents, pick_frequencies, color='red')  # Change color to red

    plt.xlabel('Agent')
    plt.ylabel('Average Pick Percentage (%)')
    plt.title('Average Pick Percentage of Agents in Sentinels')
    plt.xticks(rotation=45, ha='right')  # Rotate x-ticks for better readability
    plt.grid(axis='y')  # Add grid lines for better readability

    # Add value labels above each bar
    for bar in bars:
        yval = bar.get_height()  # Get the height of the bar
        plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.1f}%', ha='center', va='bottom', fontsize=10)

    # Show the plot
    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.show()

if __name__ == "__main__":
    plot_average_sentinel_pick_percentage()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_average_score_per_agent(input_filename='sentinels.csv'):
    # Load the sentinels CSV file
    sentinel_df = pd.read_csv(input_filename)

    # Print the columns to check for the correct column names
    print("Sentinel Data Columns:", sentinel_df.columns)

    # Check if 'avg_score' column exists
    if 'avg_score' not in sentinel_df.columns:
        print("The 'avg_score' column is missing from the Sentinel DataFrame.")
        return

    # Clean the 'avg_score' column to ensure it's numeric
    sentinel_df['avg_score'] = pd.to_numeric(sentinel_df['avg_score'], errors='coerce')  # Convert to float

    # Group by agent name and calculate the mean average score
    average_score_per_agent = sentinel_df.groupby('Agent')['avg_score'].mean().reset_index()

    # Sort by average score to get the top 20 agents
    top_20_agents = average_score_per_agent.nlargest(20, 'avg_score')

    # Sort the top 20 agents alphabetically
    top_20_agents = top_20_agents.sort_values(by='Agent')  # Sort alphabetically by agent name

    # Prepare data for plotting
    agents = top_20_agents['Agent']
    average_scores = top_20_agents['avg_score']

    # Plotting the Average Scores for Agents in Sentinels
    plt.figure(figsize=(12, 6))
    bars = plt.bar(agents, average_scores, color='red')  # Change color to red

    plt.xlabel('Agent')
    plt.ylabel('Average Score')
    plt.title('Average Score per Agent on Sentinels')
    plt.xticks(rotation=45, ha='right')  # Rotate x-ticks for better readability
    plt.grid(axis='y')  # Add grid lines for better readability

    # Add value labels above each bar
    for bar in bars:
        yval = bar.get_height()  # Get the height of the bar
        plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.1f}', ha='center', va='bottom', fontsize=10)

    # Show the plot
    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.show()

if __name__ == "__main__":
    plot_average_score_per_agent()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_top_20_radiant_pick_percentage(input_filename='radiant.csv'):
    # Load the radiant CSV file
    radiant_df = pd.read_csv(input_filename)

    # Print the columns to check for the correct column names
    print("Radiant Data Columns:", radiant_df.columns)

    # Clean the 'pick_percent' column
    radiant_df['pick_percent'] = radiant_df['pick_percent'].astype(float)  # Convert to float

    # Sort by pick_percent to get the top 20 agents
    top_20_agents = radiant_df.nlargest(20, 'pick_percent')

    # Sort the top 20 agents alphabetically
    top_20_agents = top_20_agents.sort_values(by='name')

    # Prepare data for plotting
    agents = top_20_agents['name']
    pick_frequencies = top_20_agents['pick_percent']

    # Plotting the Pick Percentages for Top 20 Agents in Radiant
    plt.figure(figsize=(12, 6))
    bars = plt.bar(agents, pick_frequencies, color='gold')  # Change color to gold

    plt.xlabel('Agent')
    plt.ylabel('Pick Percentage (%)')
    plt.title('Pick Percentage of Agents in Radiant')
    plt.xticks(rotation=45, ha='right')  # Rotate x-ticks for better readability
    plt.grid(axis='y')  # Add grid lines for better readability

    # Add value labels above each bar
    for bar in bars:
        yval = bar.get_height()  # Get the height of the bar
        plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.1f}%', ha='center', va='bottom', fontsize=10)

    # Show the plot
    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.show()

if __name__ == "__main__":
    plot_top_20_radiant_pick_percentage()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_pick_percentage_comparison(gold_file='gold.csv', radiant_file='radiant.csv'):
    # Read the gold and radiant CSV files
    gold_data = pd.read_csv(gold_file)
    radiant_data = pd.read_csv(radiant_file)

    # Print the columns to check for the correct column names
    print("Gold Data Columns:", gold_data.columns)
    print("Radiant Data Columns:", radiant_data.columns)

    # Check the data type of 'pick_percent' and convert to string if necessary
    gold_data['pick_percent'] = gold_data['pick_percent'].astype(str)
    radiant_data['pick_percent'] = radiant_data['pick_percent'].astype(str)

    # Convert 'pick_percent' to numeric, handling any errors
    gold_data['pick_percent'] = pd.to_numeric(gold_data['pick_percent'].str.replace('%', ''), errors='coerce')
    radiant_data['pick_percent'] = pd.to_numeric(radiant_data['pick_percent'].str.replace('%', ''), errors='coerce')

    # Group by agent name and calculate the mean pick percentage
    gold_pick_percentage = gold_data.groupby('name')['pick_percent'].mean().reset_index()
    radiant_pick_percentage = radiant_data.groupby('name')['pick_percent'].mean().reset_index()

    # Merge the two DataFrames on agent name
    combined_pick_percentage = pd.merge(gold_pick_percentage, radiant_pick_percentage, on='name', suffixes=('_gold', '_radiant'))

    # Prepare data for plotting
    agents = combined_pick_percentage['name']
    gold_picks = combined_pick_percentage['pick_percent_gold']
    radiant_picks = combined_pick_percentage['pick_percent_radiant']

    # Set the bar width
    bar_width = 0.35
    x = range(len(agents))

    # Plotting the pick percentages
    plt.figure(figsize=(12, 6))
    bars1 = plt.bar(x, gold_picks, width=bar_width, label='Gold Pick Percentage', color='gold')
    bars2 = plt.bar([p + bar_width for p in x], radiant_picks, width=bar_width, label='Radiant Pick Percentage', color='blue')

    plt.xlabel('Agent')
    plt.ylabel('Pick Percentage')
    plt.title('Pick Percentage Comparison Between Gold and Radiant Players')
    plt.xticks([p + bar_width / 2 for p in x], agents, rotation=45, ha='right')  # Center the x-ticks

    # Place the legend outside the plot area
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Adjust the position as needed

    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.grid(axis='y')  # Add grid lines for better readability

    # Show the plot
    plt.show()

if __name__ == "__main__":
    plot_pick_percentage_comparison()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_kd_ratio_comparison(gold_file='gold.csv', radiant_file='radiant.csv'):
    # Read the gold and radiant CSV files
    gold_data = pd.read_csv(gold_file)
    radiant_data = pd.read_csv(radiant_file)

    # Print the columns to check for the correct column names
    print("Gold Data Columns:", gold_data.columns)
    print("Radiant Data Columns:", radiant_data.columns)

    # Check if 'kd_ratio' column exists in both DataFrames
    if 'kd_ratio' not in gold_data.columns:
        print("The 'kd_ratio' column is missing from the Gold DataFrame.")
        return
    if 'kd_ratio' not in radiant_data.columns:
        print("The 'kd_ratio' column is missing from the Radiant DataFrame.")
        return

    # Group by agent name and calculate the mean kd_ratio
    gold_kd_ratio = gold_data.groupby('name')['kd_ratio'].mean().reset_index()
    radiant_kd_ratio = radiant_data.groupby('name')['kd_ratio'].mean().reset_index()

    # Merge the two DataFrames on agent name
    combined_kd_ratio = pd.merge(gold_kd_ratio, radiant_kd_ratio, on='name', suffixes=('_gold', '_radiant'))

    # Prepare data for plotting
    agents = combined_kd_ratio['name']
    gold_kd = combined_kd_ratio['kd_ratio_gold']
    radiant_kd = combined_kd_ratio['kd_ratio_radiant']

    # Set the bar width
    bar_width = 0.35
    x = range(len(agents))

    # Plotting the KD Ratios
    plt.figure(figsize=(12, 6))
    bars1 = plt.bar(x, gold_kd, width=bar_width, label='Gold KD Ratio', color='gold')
    bars2 = plt.bar([p + bar_width for p in x], radiant_kd, width=bar_width, label='Radiant KD Ratio', color='blue')

    plt.xlabel('Agent')
    plt.ylabel('KD Ratio')
    plt.title('KD Ratio Comparison Between Gold and Radiant Players')
    plt.xticks([p + bar_width / 2 for p in x], agents, rotation=45, ha='right')  # Center the x-ticks

    # Place the legend outside the plot area
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Adjust the position as needed

    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.grid(axis='y')  # Add grid lines for better readability

    # Show the plot
    plt.show()

if __name__ == "__main__":
    plot_kd_ratio_comparison()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_avg_score_comparison(gold_file='gold.csv', radiant_file='radiant.csv'):
    # Read the gold and radiant CSV files
    gold_data = pd.read_csv(gold_file)
    radiant_data = pd.read_csv(radiant_file)

    # Print the columns to check for the correct column names
    print("Gold Data Columns:", gold_data.columns)
    print("Radiant Data Columns:", radiant_data.columns)

    # Check if 'avg_score' column exists in both DataFrames
    if 'avg_score' not in gold_data.columns:
        print("The 'avg_score' column is missing from the Gold DataFrame.")
        return
    if 'avg_score' not in radiant_data.columns:
        print("The 'avg_score' column is missing from the Radiant DataFrame.")
        return

    # Group by agent name and calculate the mean avg_score
    gold_avg_score = gold_data.groupby('name')['avg_score'].mean().reset_index()
    radiant_avg_score = radiant_data.groupby('name')['avg_score'].mean().reset_index()

    # Merge the two DataFrames on agent name
    combined_avg_score = pd.merge(gold_avg_score, radiant_avg_score, on='name', suffixes=('_gold', '_radiant'))

    # Prepare data for plotting
    agents = combined_avg_score['name']
    gold_scores = combined_avg_score['avg_score_gold']
    radiant_scores = combined_avg_score['avg_score_radiant']

    # Set the bar width
    bar_width = 0.35
    x = range(len(agents))

    # Plotting the Average Scores
    plt.figure(figsize=(12, 6))
    bars1 = plt.bar(x, gold_scores, width=bar_width, label='Gold Avg Score', color='gold')
    bars2 = plt.bar([p + bar_width for p in x], radiant_scores, width=bar_width, label='Radiant Avg Score', color='blue')

    plt.xlabel('Agent')
    plt.ylabel('Average Score')
    plt.title('Average Score Comparison Between Gold and Radiant Players')
    plt.xticks([p + bar_width / 2 for p in x], agents, rotation=45, ha='right')  # Center the x-ticks

    # Place the legend outside the plot area
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Adjust the position as needed

    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.grid(axis='y')  # Add grid lines for better readability

    # Show the plot
    plt.show()

if __name__ == "__main__":
    plot_avg_score_comparison()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_top_20_radiant_average_score(input_filename='radiant.csv'):
    # Load the radiant CSV file
    radiant_df = pd.read_csv(input_filename)

    # Print the columns to check for the correct column names
    print("Radiant Data Columns:", radiant_df.columns)

    # Check the data types of the columns
    print("Data Types:\n", radiant_df.dtypes)

    # Clean the 'avg_score' column
    # Convert to string first, then remove commas and convert to float
    radiant_df['avg_score'] = radiant_df['avg_score'].astype(str).str.replace(',', '').astype(float)  # Remove commas and convert to float

    # Sort by avg_score to get the top 20 agents
    top_20_agents = radiant_df.nlargest(20, 'avg_score')

    # Sort the top 20 agents alphabetically
    top_20_agents = top_20_agents.sort_values(by='name')

    # Prepare data for plotting
    agents = top_20_agents['name']
    avg_scores = top_20_agents['avg_score']

    # Plotting the Average Scores for Top 20 Agents in Radiant
    plt.figure(figsize=(12, 6))
    bars = plt.bar(agents, avg_scores, color='gold')  # Change color to gold

    plt.xlabel('Agent')
    plt.ylabel('Average Score')
    plt.title('Average Score of Agents in Radiant')
    plt.xticks(rotation=45, ha='right')  # Rotate x-ticks for better readability
    plt.grid(axis='y')  # Add grid lines for better readability

    # Add value labels above each bar
    for bar in bars:
        yval = bar.get_height()  # Get the height of the bar
        plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.1f}', ha='center', va='bottom', fontsize=10)

    # Show the plot
    plt.tight_layout()  # Adjust layout to make room for rotated labels
    plt.show()

if __name__ == "__main__":
    plot_top_20_radiant_average_score()